In [ ]:
import pandas as pd
import numpy as np
import xgboost
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import warnings
import lightgbm as lgb
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 80) 
pd.set_option('display.max_rows', 100) 
%matplotlib inline

In [2]:
data = pd.read_csv('../data/demand_anonymized_20170802.csv', sep=';', 
                   parse_dates=['First_MAD', 'Month'])
eval_data = pd.read_csv('../data/eval.csv', parse_dates=['date'])

In [3]:
enc = LabelEncoder()
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = enc.fit_transform(data[col].fillna('nan'))
        if col in eval_data.columns:
            eval_data[col] = enc.transform(eval_data[col])

In [4]:
month_min = data.Month.min()
def get_month(month):
    return (month.year - month_min.year)*12 + (month.month - month_min.month)

In [5]:
data['group'] = data.Material*10 + data.SalOrg
data['month'] = data.Month.apply(get_month)

In [6]:
eval_data['group'] = eval_data.Material*10 + eval_data.SalOrg

In [7]:
data['group_month'] = data.group*100 + data.month

In [8]:
unique_group = sorted(data.group.unique())
month_num = data.month.max() + 1

In [9]:
table_index = pd.DataFrame()
table_index['group'] = unique_group * (month_num)

month = []
for i in range(month_num):
    month += [i]*len(unique_group)
table_index['month'] = month

table_index['group_month'] = table_index.group*100 + table_index.month

In [10]:
data_new = table_index[['group_month']].merge(data, on='group_month', how='left')

In [11]:
data_new.OrderQty = data_new.OrderQty.fillna(0)

In [12]:
data_new.group = (data_new.group_month / 100).apply(int)
data_new.month = data_new.group_month % 100

In [13]:
def get_y(data_new,  month_min, month_max):
    table = data_new[(data_new.month >= month_min) & (data_new.month <= month_max)]
    y = table.groupby('group_month').OrderQty.sum()
    y_new = [[], [], []]
    for i, t in enumerate(y):
        y_new[i % 3].append(t)
    return y_new[0] + y_new[1] + y_new[2]

In [14]:
def regression(x, shift):
    if (x == 0).sum() == len(x):
        return 0
    if len(x) < 2:
        return 0
    A = np.vstack([np.arange(0, len(x)), np.ones(len(x))]).T
    return np.linalg.inv(A.T.dot(A)).dot(A.T).dot(x).dot([len(x) + shift - 1, 1])

In [15]:
def hint(x):
    a = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
    a = a / a.sum()
    return a.dot(x[-len(a):])

In [16]:
def hint2(x):
    a = 0.9**np.arange(10, -1, -1)
    a = a / a.sum()
    return a.dot(x[-len(a):])

In [17]:
def hint3(x):
    a = 0.8**np.arange(10, -1, -1)
    a = a / a.sum()
    return a.dot(x[-len(a):])

In [18]:
def hint4(x):
    a = 0.6**np.arange(10, -1, -1)
    a = a / a.sum()
    return a.dot(x[-len(a):])

In [19]:
def num_zeros(x):
    n = len(x) - 1
    s = 0
    for i in range(n, n - 50, -1):
        if x[i] == 0:
            s += 1
        else:
            break
    return s

In [20]:
data_new = data_new.fillna(-1)

In [22]:
categ_col = ['LogABC', 'ItemCat', 'PL', 'SubFct', 'MktABC', 'Manufacturer', 'ORIGINAL_SUPPLIER', 'Plant',  
             'Gamma',  'Business', 'DP_FAMILY_CODE',
             'PRODUCT_STATUS', 'SUBRANGE']

In [23]:
col_unique = {}
for col in categ_col:
    col_unique[col] = data_new[col].unique()

In [24]:
def get_X1(sequence, shift, table):
    
    X = pd.DataFrame(index=unique_group)
    
    X['last_month'] = sequence.apply(lambda x: x[-1])
    X['last_4_month'] = sequence.apply(lambda x: np.mean(x[-4:]))
    X['last_8_month'] = sequence.apply(lambda x: np.mean(x[-8:]))
    X['last_16_month'] = sequence.apply(lambda x: np.mean(x[-16:]))

    X['median1'] = sequence.apply(lambda x: np.median(x[-4:]))
    X['median2'] = sequence.apply(lambda x: np.median(x[-8:]))
    X['median3'] = sequence.apply(lambda x: np.median(x[-16:]))
    X['median4'] = sequence.apply(lambda x: np.median(x[-32:]))

    X['num_zeros'] = sequence.apply(lambda x: num_zeros(x))
    X['num_zeros2'] = sequence.apply(lambda x: (np.array(x[-5:]) == 0).sum())
    X['num_zeros2'] = sequence.apply(lambda x: (np.array(x[-10:]) == 0).sum())
    X['num_zeros3'] = sequence.apply(lambda x: (np.array(x[-20:]) == 0).sum())
    X['num_zeros4'] = sequence.apply(lambda x: (np.array(x[-40:]) == 0).sum())
    X['num_zeros4'] = sequence.apply(lambda x: (np.array(x) == 0).mean())
    
    
    X['hint'] = sequence.apply(lambda x: hint(x))
    X['hint2'] = sequence.apply(lambda x: hint2(x))
    X['hint3'] = sequence.apply(lambda x: hint3(x))
    X['hint4'] = sequence.apply(lambda x: hint4(x))
    
    X['f1'] = X.hint*0.9 + X.last_month*0.1 
    
    for col in ['LogABC', 'ItemCat', 'SubFct', 'MktABC', 'Gamma', 'Business',
                'DP_FAMILY_CODE', 'PRODUCT_STATUS', 'SUBRANGE']:
        for value in col_unique[col]:
            X[col + str(value)] = (table[col] == value).groupby(table.group).mean()
            
    for col in ['Name_Of_Competitor',
               'COMP_PRICE_AVG', 'COMP_PRICE_MIN', 'COMP_PRICE_MAX', 'PRICE', 'NEAREST_COMP_PRICE_MIN',
               'NEAREST_COMP_PRICE_MAX',]:
        X[col] = table[col].groupby(table.group).median()
    
    #X['SalOrg'] = (np.array(sequence.index) / 10).astype(int)
    
    return X

def get_X2(sequence, shift, table):
    X = pd.DataFrame(index=unique_group)
    
    X['regression'] = sequence.apply(lambda x: regression(np.array(x), shift))
    
    X['year_ago_diff'] = sequence.apply(lambda x: x[-12 + shift -1]) - sequence.apply(lambda x: x[-13 + shift - 1])
    X['month_-1year'] = sequence.apply(lambda x: x[-12 + shift - 1])
    X['month_-2year'] = sequence.apply(lambda x: x[-24 + shift - 1])
    X['month_-2year'] = sequence.apply(lambda x: x[-36 + shift - 1])
    X['month_diff'] = X['month_-1year'] - X['month_-2year']
    X['good'] = sequence.apply(lambda x: x[-12 + shift - 1] + (x[-1] - x[-13]))
    X['month_mean_year'] = sequence.apply(lambda x: np.mean([x[-36 + shift - 1], x[-24 + shift - 1], x[-12 + shift - 1]]))
    
    X['shift'] = shift
    
    return X
    
def create_X(data_new, month_min):
    
    table = data_new[data_new.month < month_min]
    
    tmp = table.groupby('group_month').OrderQty.sum()
    group_tmp = (np.array(tmp.index) / 100 ).astype(int)
    sequence = tmp.groupby(group_tmp).apply(list)

    X = get_X1(sequence, 1, table)
    X1 = get_X2(sequence, 1, table).join(X)
    X2 = get_X2(sequence, 2, table).join(X)
    X3 = get_X2(sequence, 3, table).join(X)
    
    return pd.concat([X1, X2, X3])

In [ ]:
def get_mae(row, y):
    return (row - y).abs().mean()

In [ ]:
%%time
Xtest = create_X(data_new, 63)

In [ ]:
%%time
X = [create_X(data_new, 60 - i) for i in range(15)]
y = [get_y(data_new, 60 - i, 62 - i) for i in range(15)]
y = [pd.Series(i) for i in y]

In [301]:
ids = eval_data[:int(len(eval_data)/3)].sort_values('group').ID.values

In [ ]:
model = xgboost.XGBClassifier(n_estimators=40, max_depth=15, subsample=0.9, learning_rate=0.1)

In [ ]:
model.fit(pd.concat(X), pd.concat(y))
p = model.predict(Xtest)

In [ ]:
submission = pd.DataFrame()
submission['demand'] = p
submission['ID'] = list(ids) + list(ids + len(ids)) + list(ids + 2*len(ids))
submission.demand[submission.demand < 0] = 0
submission.to_csv('res6.csv', index=False)

In [ ]:
9.82201
9.73349
9.61947 8
9.34679 10


In [63]:
pd.concat(X[1:]).shape

(2204532, 52)

In [64]:
pd.concat(X).shape

(2320560, 52)

In [ ]:
pd.concat(X[1:]).to_csv('Xtrain.csv')
pd.concat(X).to_csv('X.csv')
X[0].to_csv('Xval.csv')
Xtest.to_csv('Xtest.csv')

pd.concat(y[1:]).to_csv('ytrain.csv')
pd.concat(y).to_csv('y.csv')
y[0].to_csv('yval.csv')